In [196]:
import numpy as np
import pandas as pd
import re
import pickle
from pprint import pprint
import datetime
import math
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import json
from bson import json_util

from mongoengine import *

In [164]:
from flask import Flask,jsonify, request, send_from_directory, url_for
app = Flask(__name__)
import pickle
import numpy as np
from itertools import chain

In [165]:
conn = connect("local",host="mongodb://localhost:27017",alias='default')

if(0):
    disconnect('mongo://mongo:27017')

In [167]:
str_menu=pickle.load(open('full_menu_main.pkl','rb'))
if(0):
    pickle.dump(str_menu,open('half_menu_main.pkl','wb'))
restautant_id = 'BNGHSR0001'

final_list_json=[]

In [177]:
str_menu['price']=str_menu['price'].apply(lambda x: x.split('/')[0])

In [169]:
def chainer(string):
    return list(chain.from_iterable(string.str.split('/')))

def str_2(number):
    string = str(number)
    if(len(string)==1):
        return '0'+string
    else:
        return string

In [179]:
final_list_json=[]
for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['foodlist']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['foodlist'].append({})
            final_list_json[n]['sub_category'][p]['foodlist'][q]['name']= food_dict[1]['name']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['price']=food_dict[1]['price']
        

In [180]:
pickle.dump(final_list_json,open('jason_bad.pkl','wb'))

final_list_json=pickle.load(open('jason_bad.pkl','rb'))

In [287]:

class TableOrder(Document):
    pass


class User(Document):
    dine_in_history = DynamicField()
    current_table_id = StringField()
    personal_cart = ListField(ReferenceField(TableOrder))
    meta = {'allow_inheritance': True}


class TempUser(User):
    pass


class RegisteredUser(User):
    name = StringField(required=True)
    email_id = StringField(required=True)
    phone_no = StringField()
    tempuser_ob = ReferenceField(TempUser)


class TempUser(User):
    unique_id = StringField(required=True)
    reguser_ob = ReferenceField(RegisteredUser)


class Assistance(Document):
    pass


class Server(Document):
    name = StringField()
    assistance_history = MapField(ListField(ReferenceField(Assistance)))
    order_history = MapField(ListField(ReferenceField(TableOrder)))

    def to_mymongo(self):
        data = self.to_mongo()

        return data

class Assistance(Document):
    types = ['water', 'help', 'cutlery', 'tissue', 'cleaning', 'menu', 'ketchup']
    table = StringField()
    user = ReferenceField(User)
    assistance_type = StringField(choices=types)
    timestamp = DateTimeField()
    accepted_by = ReferenceField(Server, default=None)
    meta = {'strict': False}

    def to_mymongo(self):
        data = self.to_mongo()
        data['timestamp'] = str(data['timestamp'])
        data['table'] = Table.objects(assistance_reqs__in=[self.id])[0].name

        return data

    def to_json(self):
        data = self.to_mongo()
        data['timestamp'] = str(data['timestamp'])
        data['table'] = Table.objects(assistance_reqs__in=[self.id])[0].name
        return json_util.dumps(data)


class FoodOptionsMod(EmbeddedDocument):
    size_field = DictField()


class FoodItemMod(EmbeddedDocument):
    food_id=StringField()
    name = StringField()
    description = StringField()
    price = StringField()
    instructions = StringField()
    quantity = IntField()
    status = StringField(choices=['queued','cooking','completed'])
    foodoptions = EmbeddedDocumentField(FoodOptionsMod)
    

class Order(Document):
    placedby = ReferenceField(User)
    foodlist = ListField(EmbeddedDocumentField(FoodItemMod))
    status = StringField(choices=['queued','juststarted','cooking','almostdone','completed'],default='queued')
    
    def fetch_fooditem(self,food_id):
        for fooditem in self.foodlist:
            if(fooditem.food_id==food_id):
                return fooditem.to_json()
        return "Food item not found"
        
    
class TableOrder(Document):
    table =StringField()
    orders = ListField(ReferenceField(Order))
    status = StringField(choices=['queued','juststarted','cooking','almostdone','completed'],default='queued')
    timestamp = DateTimeField(default=datetime.datetime.now())

    def to_mymongo(self):
        data = self.to_mongo()
        data['timestamp'] = str(data['timestamp'])
        for key, order in enumerate(self.orders):
            data['orders'][key] = json_util.loads(self.orders[key].to_json())

        return data
    

    def to_json(self):
        data = self.to_mongo()
        data['timestamp'] = str(data['timestamp'])
        for key, order in enumerate(self.orders):
            data['orders'][key] = json_util.loads(self.orders[key].to_json())
        return json_util.dumps(data)


class Table(Document):
    name = StringField(required=True)
    seats = IntField(required=True)
    servers = ListField(ReferenceField(Server))
    users = ListField(ReferenceField(User))
    nofusers = IntField()
    qd_tableorders = ListField(ReferenceField(TableOrder))
    cook_tableorders = ListField(ReferenceField(TableOrder))
    com_tableorders = ListField(ReferenceField(TableOrder))
    assistance_reqs = ListField(ReferenceField(Assistance))
    meta = {'strict': False}

    def to_mymongo(self):
        data = self.to_mongo()
        for key, tableorder in enumerate(self.qd_tableorders):
            data['qd_tableorders'][key] = self.qd_tableorders[key].to_mymongo()
        for key, tableorder in enumerate(self.cook_tableorders):
            data['cook_tableorders'][key] = self.cook_tableorders[key].to_mymongo()
        for key, tableorder in enumerate(self.com_tableorders):
            data['com_tableorders'][key] = self.com_tableorders[key].to_mymongo()
        for key, ass_req in enumerate(self.assistance_reqs):
            data['assistance_reqs'][key] = self.assistance_reqs[key].to_mymongo()
            

        return data


class FoodOptions(DynamicDocument):
    size_field = DictField()
    best_paired_with = ListField(StringField())


class FoodItem(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    price = StringField(required=True)
    foodoptions = ReferenceField(FoodOptions)

    def to_mymongo(self):
        data = self.to_mongo()
        if (self.foodoptions):
            data['foodoptions'] = self.foodoptions.to_mongo()

        return data

    def to_json(self):
        data = self.to_mongo()
        if (self.foodoptions):
            data['foodoptions'] = self.foodoptions.to_mongo()

        return json_util.dumps(data)


class SubCategory(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    foodlist = ListField(ReferenceField(FoodItem), required=True)

    def to_mymongo(self):
        data = self.to_mongo()
        for key, fooditem in enumerate(self.foodlist):
            data['foodlist'][key] = self.foodlist[key].to_mymongo()

        return data


class MainCategory(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    sub_category = ListField(ReferenceField(SubCategory), required=True)

    def to_mymongo(self):
        data = self.to_mongo()
        for key, sub_cat in enumerate(self.sub_category):
            data['sub_category'][key] = self.sub_category[key].to_mymongo()

        return data


class Restaurant(Document):
    name = StringField(required=True)
    restaurant_id = StringField(required=True)
    menu = ListField(ReferenceField(MainCategory), required=True)
    address = StringField()
    tables = ListField(ReferenceField(Table))
    servers = ListField(ReferenceField(Server))

    def to_json(self):
        data = self.to_mongo()
        for key, sub_cat in enumerate(self.menu):
            data['menu'][key] = self.menu[key].to_mymongo()
        for key, server in enumerate(self.servers):
            data['servers'][key] = self.servers[key].to_mymongo()
        for key, table in enumerate(self.tables):
            data['tables'][key] = self.tables[key].to_mymongo()

        return json_util.dumps(data)


# Generating data

In [221]:
if(1):
    FoodItem.drop_collection()
    SubCategory.drop_collection()
    FoodOptions.drop_collection()
    MainCategory.drop_collection()
    Restaurant.drop_collection()
    Table.drop_collection()
    TempUser.drop_collection()
    RegisteredUser.drop_collection()
    Server.drop_collection()
    Assistance.drop_collection()
    TableOrder.drop_collection()
    Order.drop_collection()
    User.drop_collection()

    main_cat_list = []
    for p,main_cat in enumerate(final_list_json):
        sub_cat_list = []
        for q,sub_cat in enumerate(main_cat['sub_category']):
            food_list = []
            for r,food in enumerate(sub_cat['foodlist']):
                food_list.append(FoodItem(name=food['name'],description = food['description'],price=food['price']).save())
            sub_cat_list.append(SubCategory(name=sub_cat['name'],description=sub_cat['description'],foodlist = food_list).save())
        main_cat_list.append(MainCategory(name=main_cat['name'],description = main_cat['description'],sub_category = sub_cat_list).save())

    houseofcommons=Restaurant(name = 'House of Commons',restaurant_id='BNGHSR0001',menu = main_cat_list,address='')
    houseofcommons.save()

    table_list=[]
    for n in range(1,16):
        table_list.append(Table(name='table'+str(n),seats = 6,nofusers=0).save().to_dbref())

    Restaurant.objects[0].update(set__tables=table_list)
    akshay = TempUser(unique_id = "hfirnivnhhwocn34534no34n34r")

    akshay.save()

    RegisteredUser(name='Akshay',email_id='akshay.dn5@gmail.com',phone_no='8660961089').save()

    RegisteredUser.objects(name='Akshay',email_id='akshay.dn5@gmail.com',phone_no='8660961089').update(set__tempuser_ob=akshay.to_dbref())

    user_scan(str(table_list[2].id),'hidnfvidfkvmsdmv')

    user_scan(str(table_list[2].id),'hidnfvidvdff1kvmsadmv')

    user_scan(str(table_list[2].id),'hidnfvidvdffaskvmsadmv')

    user_scan(str(table_list[2].id),'hidnfvidvdffvf1kvmsadmv')

    user_scan(str(table_list[2].id),'',email_id='akshay.dn5@gmail.com')

In [222]:
#RANDOM Functions
def random_table():
    return all_table_list[np.random.randint(len(all_table_list))]
def random_user():
    return user_list[np.random.randint(len(user_list))]
def random_food():
    return all_food_list[np.random.randint(len(all_food_list))]
def random_food_list():
    randomfl=[]
    for i in range(np.random.randint(2,9)):
        randomfl.append(random_food())
    return list(set(randomfl))

In [223]:
def user_scan(table_id,unique_id,email_id='dud'):
    scanned_table=Table.objects.get(id=table_id)
    if(email_id=='dud'):
        tempuser = TempUser.objects.filter(unique_id=unique_id)
        if(len(tempuser)>0):
            tempuser[0].update(set__current_table_id=str(scanned_table.id))
            scanned_table.update(push__users=tempuser[0].to_dbref())
        else:
            tempuser= TempUser(unique_id=unique_id,current_table_id=str(scanned_table.id)).save()
            scanned_table.update(push__users=tempuser.to_dbref())
        scanned_table.update(inc__nofusers=1)
    else:
        reg_user=RegisteredUser.objects.filter(email_id=email_id)[0]
        scanned_table.update(push__users=reg_user)
        reg_user.update(set__current_table_id=str(scanned_table.id))
        scanned_table.update(inc__nofusers=1)

In [224]:
user_list=[]
for obj in User.objects:
    user_list.append(str(obj.id))

all_table_list=[]
for obj in Table.objects:
    all_table_list.append(str(obj.id))
    
all_food_list=[]
for obj in FoodItem.objects:
    all_food_list.append(str(obj.id))

In [225]:
try:
    if(names):
        pass
except:
    names=pickle.load(open('indian_names.pkl','rb'))
    girls_name=names[0]
    boys_name=names[1]

In [226]:
if(1):
    total = {}
    sers=[]
    tabs=[]
    for n,table in enumerate(Table.objects):
    #     print(math.floor(n/1.5))
        sers.append(math.floor(n/1.5))
        tabs.append(n)
        if((n+1)%3==0):
            total[tuple(set(sers))]=tabs
            sers=[]
            tabs=[]


    if(1):#assiging tables to servers
        for i in range(10):
            if(i<5):
                server=Server(name=girls_name[np.random.randint(len(girls_name))]).save()
                Restaurant.objects[0].update(push__servers=server.to_dbref())
            else:
                server=Server(name=boys_name[np.random.randint(len(boys_name))]).save()
                Restaurant.objects[0].update(push__servers=server.to_dbref())

    for key in total.keys():
        for tab in total[key]:
            Table.objects[tab].update(push__servers=Server.objects[key[0]].to_dbref())

        for tab in total[key]:
            Table.objects[tab].update(push__servers=Server.objects[key[1]].to_dbref())

# Otehrs

In [192]:
def food_embed(food_dict):
    json_dict=json_util.loads(FoodItem.objects(id=food_dict['food_id']).exclude('id')[0].to_json())
    json_dict['food_id']=food_dict['food_id']
    json_dict['quantity']=food_dict['quantity']
    json_dict['instructions']=food_dict['instructions']
    json_dict['status']='queued'
    return json_util.dumps(json_dict)


def c_food_dict(food_id):
    f_dict = {}
    f_dict['food_id']=food_id
    f_dict['quantity']=int(np.random.choice([1,1,1,1,2,2,3]))
    f_dict['instructions'] = ['Cook with love' if np.random.randint(2) else 'no'][0]
    return f_dict
def nprand(a,b=-1):
    if(b==-1):
        return np.random.randint(a)
    else:
        return np.random.randint(a,b)

In [193]:
def generate_order():
    input_order={}
    input_order['table'] = random_table()
    input_order['orders'] = []
    for n in range(np.random.randint(1,5)):
        input_order['orders'].append({})
        input_order['orders'][n]['placedby']=random_user()
        input_order['orders'][n]['foodlist'] = [c_food_dict(v) for v in random_food_list()]
    return input_order

def order_placement(input_order):
    ordered_table = Table.objects.get(id=input_order['table'])
    tableorder = TableOrder(table=ordered_table.name,timestamp=datetime.datetime.now())
    for order in input_order['orders']:
        foodlist=[FoodItemMod.from_json(food_embed(food_dict)) for food_dict in order['foodlist']]
        tableorder.orders.append(Order(placedby=User.objects.get(id=order['placedby']).to_dbref(),foodlist=foodlist).save().to_dbref())
        tableorder.timestamp=datetime.datetime.now()
        tableorder.save()
    ordered_table.update(push__qd_tableorders=tableorder.to_dbref())
    return TableOrder.objects.get(id=tableorder.id).to_json()

def generate_asstype():
    assist_input = {'table':random_table(),'user':random_user(),'assistancetype':Assistance.types[np.random.randint(len(Assistance.types))]}
    return assist_input

def assistance_req(assist_input):
    curr_table=Table.objects.get(id=assist_input['table'])
    curr_ass=Assistance(user = User.objects.get(id=assist_input['user']).to_dbref())
    curr_ass.assistance_type=assist_input['assistancetype']
    curr_ass.timestamp=datetime.datetime.now()
    curr_ass.save()
    curr_table.update(push__assistance_reqs=curr_ass.to_dbref())
    return curr_ass

def send_assistance_req(assist_id):
    table=Table.objects(assistance_reqs__in=[assist_id])[0]
    for server in table.servers:
        if(np.random.randint(3)):
            time.sleep(1)
            Assistance.objects.get(id=assist_id).update(set__accepted_by=server.to_dbref())
            return server.name
    time.sleep(3)
    for server in Server.objects:
        if(np.random.randint(3)):
            time.sleep(1)
            Assistance.objects.get(id=assist_id).update(set__accepted_by=server.to_dbref())
            return server.name

In [194]:
TableOrder.drop_collection()

Order.drop_collection()

In [197]:
send_assistance_req(str(assistance_req(generate_asstype()).id))

'Mohini'

In [198]:

def non_completed(clas_objects,cooking,skip=0):
    if(cooking):
        for obj in clas_objects:
            if(obj.status=='queued'):
                if(skip==0):
                    return obj
                else:
                    skip=0
                    continue
        return 'all_cooking'
    else:
        for obj in clas_objects:
            if(obj.status=='cooking'):
                if(skip==0):
                    return obj
                else:
                    skip=0
                    continue
        return 'all_completed'

def food_status_check(clas_objects,skip=0):
    for obj in clas_objects:
        if(obj.status=='queued'):
            if(skip==0):
                return obj
            else:
                skip=0
                continue
    return 'all_cooking'

def food_status_check_cook(clas_objects,skip=0):
    for obj in clas_objects:
        if(obj.status=='cooking'):
            if(skip==0):
                return obj
            else:
                skip=0
                continue
    return 'all_completed'

def pick_order():
    if(True):
        tableorder=non_completed(TableOrder.objects,True)
        order=non_completed(tableorder.orders,True)
        if(not isinstance(order,Order)):
            TableOrder.objects.get(id=tableorder.id).update(set__status='cooking')
            print('changeing tableorder status')
            return pick_order()
        
        food_ob=food_status_check(order.foodlist)
        if(isinstance(food_ob,FoodItemMod)):
            food_id=food_ob.food_id
            return (tableorder.id,order.id,food_id)
        elif(food_ob=='all_cooking'):
            Order.objects.get(id=order.id).update(set__status='cooking')
            print('changeing order status')
            return pick_order()
    else:
        tableorder=non_completed(TableOrder.objects,1)
        order=non_completed(tableorder.orders)
        food_id=food_status_check_cook(order.foodlist).food_id
        
        return (tableorder.id,order.id,food_id)

def pick_order2():
    if(True):
        tableorder=non_completed(TableOrder.objects,False)
        order=non_completed(tableorder.orders,False)
        if(not isinstance(order,Order)):
            TableOrder.objects.get(id=tableorder.id).update(set__status='completed')
            print('changeing tableorder status')
            return pick_order()
        
        food_ob=food_status_check_cook(order.foodlist)
        if(isinstance(food_ob,FoodItemMod)):
            food_id=food_ob.food_id
            return (tableorder.id,order.id,food_id)
        elif(food_ob=='all_completed'):
            Order.objects.get(id=order.id).update(set__status='completed')
            print('changeing order status')
            return pick_order()
    
        
    else:
        tableorder=non_completed(TableOrder.objects,1)
        order=non_completed(tableorder.orders)
        food_id=food_status_check(order.foodlist).food_id
        
        return (tableorder.id,order.id,food_id)

In [199]:
def validate_for_order(order_id,cooking=True):
    if(cooking):
        for food_item in Order.objects.get(id=order_id).foodlist:
            if(food_item.status=='cooking'):
                continue
            else:
                return False
        return True
    else:
        for food_item in Order.objects.get(id=order_id).foodlist:
            if(food_item.status=='completed'):
                continue
            else:
                return False
        return True

def validate_for_tableorder(order_id,cooking=True):
    if(cooking):
        for order in TableOrder.objects.get(id=order_id).orders:
            if(order.status=='cooking'):
                continue
            else:
                return False
        return True
    else:
        for order in TableOrder.objects.get(id=order_id).orders:
            if(order.status=='completed'):
                continue
            else:
                return False
        return True

def order_status_cooking(status_tuple):
    tableorder=TableOrder.objects.get(id=status_tuple[0])
    order=Order.objects.get(id=status_tuple[1])
    food_json_dict=json_util.loads(order.fetch_fooditem(status_tuple[2]))
    food_json_dict['status']='cooking'
    Order.objects.get(id=status_tuple[1]).update(pull__foodlist__food_id=FoodItemMod(food_id=status_tuple[2]).food_id)
    Order.objects.get(id=status_tuple[1]).update(push__foodlist=FoodItemMod.from_json(json_util.dumps(food_json_dict)))
    if(validate_for_order(status_tuple[1],True)):
        Order.objects.get(id=status_tuple[1]).update(set__status='cooking')
    if(validate_for_tableorder(status_tuple[0],True)):
        TableOrder.objects.get(id=status_tuple[0]).update(set__status='cooking')
    return "Done"

def order_status_completed(status_tuple):
    tableorder=TableOrder.objects.get(id=status_tuple[0])
    order=Order.objects.get(id=status_tuple[1])
    food_json_dict=json_util.loads(order.fetch_fooditem(status_tuple[2]))
    food_json_dict['status']='completed'
    Order.objects.get(id=status_tuple[1]).update(pull__foodlist__food_id=FoodItemMod(food_id=status_tuple[2]).food_id)
    Order.objects.get(id=status_tuple[1]).update(push__foodlist=FoodItemMod.from_json(json_util.dumps(food_json_dict)))
    if(validate_for_order(status_tuple[1],False)):
        Order.objects.get(id=status_tuple[1]).update(set__status='completed')
    if(validate_for_tableorder(status_tuple[0],False)):
        TableOrder.objects.get(id=status_tuple[0]).update(set__status='completed')
    return "Done"

In [200]:
TableOrder.drop_collection()

Order.drop_collection()

In [235]:
order_placement(generate_order())

'{"_id": {"$oid": "5e8804e434ea6fd66b13e647"}, "table": "table12", "orders": [{"_id": {"$oid": "5e8804e434ea6fd66b13e646"}, "placedby": {"$oid": "5e8804c634ea6fd66b13e622"}, "foodlist": [{"food_id": "5e8804c534ea6fd66b13e5d6", "name": "SPAGHETTI AGLIO OLIO", "description": "NA", "price": "300", "instructions": "Cook with love", "quantity": 1, "status": "queued"}, {"food_id": "5e8804c534ea6fd66b13e5d7", "name": "FIERY PENNE PASTA", "description": "NA", "price": "300", "instructions": "no", "quantity": 1, "status": "queued"}, {"food_id": "5e8804c534ea6fd66b13e57a", "name": "TANDOORI CHICKEN CHAAT", "description": "Clay Oven Roasted Chicken Blended With Classic Spices", "price": "220", "instructions": "no", "quantity": 3, "status": "queued"}, {"food_id": "5e8804c534ea6fd66b13e58b", "name": "GARLIC BREAD", "description": "Garlic Butter Flavoured Bread Topped With Cheese", "price": "190", "instructions": "no", "quantity": 2, "status": "queued"}, {"food_id": "5e8804c534ea6fd66b13e5a7", "name

In [259]:
status_tuple=pick_order()

order_status_cooking(status_tuple)

'Done'

In [264]:
status_tuple=pick_order2()

order_status_completed(status_tuple)

'Done'

# Returning restaurant object

In [285]:
pprint(json_util.loads(Restaurant.objects.exclude('menu','servers')[0].to_json())['tables'][5]['qd_tableorders'])

[{'_id': ObjectId('5e8804e234ea6fd66b13e63b'),
  'orders': [{'_id': ObjectId('5e8804e234ea6fd66b13e63a'),
              'foodlist': [{'description': 'Tender Cottage Cheese Wrapped In '
                                           'Fragrant Pandan Leaves, With '
                                           'Chilli Coriander Sauce',
                            'food_id': '5e8804c534ea6fd66b13e592',
                            'instructions': 'Cook with love',
                            'name': 'PANDAN LEAF COTTAGE CHEESE',
                            'price': '280',
                            'quantity': 1,
                            'status': 'queued'},
                           {'description': 'Classic Indian Style With Bone '
                                           'Deep fried Chicken Served With '
                                           'Mint Chutney',
                            'food_id': '5e8804c534ea6fd66b13e5a4',
                            'instructions': 'no',
          

In [217]:
Restaurant.objects.exclude('menu')

[<Restaurant: Restaurant object>]

In [2169]:
if(0):
    order=Order.objects.get(id='5e8097106a2fad6b020c61d4')

    order.update(pull__foodlist__food_id=FoodItemMod(food_id='5e7f298b6a2fad6b020c6033').food_id)

    Order.objects.get(id='5e8097106a2fad6b020c61d4').update(push__foodlist=FoodItemMod.from_json(json_util.dumps(food)))

    food=json_util.loads(Order.objects.get(id='5e8097106a2fad6b020c61d4').fetch_fooditem('5e7f298b6a2fad6b020c6033'))

    food['status']='completed'


    food

In [2186]:
json_util.loads(Order.objects.get(id=status_tuple[1]).to_json())

{'_id': ObjectId('5e8097116a2fad6b020c61d8'),
 'placedby': ObjectId('5e7f298d6a2fad6b020c6053'),
 'foodlist': [{'food_id': '5e7f298b6a2fad6b020c5ff7',
   'name': 'CHEESE DYNABITES',
   'description': 'Classic Cheese Balls Served with New Orleans Dip',
   'price': '220',
   'instructions': 'no',
   'quantity': 2,
   'status': 'cooking'},
  {'food_id': '5e7f298b6a2fad6b020c5ffb',
   'name': 'HOC SALAD (VEG)',
   'description': 'Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing',
   'price': '180',
   'instructions': 'Cook with love',
   'quantity': 2,
   'status': 'completed'},
  {'food_id': '5e7f298b6a2fad6b020c6020',
   'name': 'JIANG CHILLI CHICKEN',
   'description': 'Chicken Tossed With Shredded Ginger And Chilli In Jiang Style',
   'price': '260',
   'instructions': 'no',
   'quantity': 1,
   'status': 'completed'}],
 'status': 'pending'}

# Flask shit

In [7]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
    
app = Flask(__name__)
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('my event')
def test_message(message):
    emit('response', {'data':'Connected with me, sock'})
    
@socketio.on('connect', namespace='/adhara')
def connect():
    print('connected')
    emit('connect', {'data':'Connected with me, sock'})


if __name__ == '__main__':
    socketio.run(app)

KeyboardInterrupt
2020-03-25T10:51:57Z


KeyboardInterrupt: 

In [ ]:
from flask import Flask
from flask_sockets import Sockets
from gevent import monkey
monkey.patch_all()

app = Flask(__name__)
sockets = Sockets(app)


@sockets.route('/echo')
def echo_socket(ws):
    while not ws.closed:
        message = ws.receive()
        print(message)
        ws.send(message)


@app.route('/')
def hello():
    return 'Hello World!'


if __name__ == "__main__":
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    server = pywsgi.WSGIServer(('', 5000), app, handler_class=WebSocketHandler)
    server.serve_forever()

yugh
yughgjkj


KeyboardInterrupt
2020-03-24T19:06:45Z


In [169]:
classes_json=json.loads(Restaurant.objects[0].to_json())

In [170]:
with open('json_with_class.json','w') as f:
    f.write(str(classes_json))

# Pages

In [14]:
pages={}

In [15]:
pages[0]="""$BAR BITES

NACHOS (Veg/Chicken)

Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream 

190/230

CHEESE MASALA PAPAD

Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese

160

H.O.C SIGNATURE MASALA PEANUTS

A Favorful Version Of Regular Peanuts Tossed In A Variety of Spices 

160

SPICY TRICOLOUR CHAAT

Fresh Peas, Corn Kernels & Black Gram Tossed With Chillies & Sirka onions

180

CHATPATA CHANA CHAAT

A Delectable Blend Of Bengal Gram And A Tangy Mixture

160

TANDOORI CHICKEN CHAAT

Clay Oven Roasted Chicken Blended With Classic Spices

220

CHEESE DYNABITES

Classic Cheese Balls Served with New Orleans Dip

220

$SALADS

GREEN SALAD

Traditional Green Cuts With Mint Yoghurt Dip

170

HOC SALAD (Veg/Chicken)

Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing

180/210

WATERMELON & FETA

An All Time Classic - With Balsamic Glaze

210

CAESAR SALAD (Veg/Chicken)

Crunchy Ice Berg Lettuce, Parmesan With the Classic Caesar Dressing

190/230"""

In [16]:
pages[1]="""$VEG STARTERS

CHEESE & CORN TRIANGLE (new)

Shredded Cheese, Corn & Seasonings Wrapped In Wonton Sheet & Deep Fried

220

CHAMPIGNONS FARCIS (new)

Sauteed Button Mushroom Stuffed With Cheese & Herbs

230

CHILLI CHEESE CIGARA (new)

Deep Fried Rolls With Cheese, Chill & Herbs

220

HOC CHIP & DIP BASKET

Combination Of Onion Rings, Jalapeno Cheese Balls, Greek Potato Wedges, Habanero Green Chili Fries

390

FRENCH FRIES

French Fries Tossed With Hoc Signature Cajun Spice Seasoning

190

ONION RINGS

Extra Crispy Onion Rings With A Spicy Salsa

190

MUSHROOM LOADED FRIES

Cheesy French Fries Topped With Mushrooms And Herbs

230

GARLIC BREAD

Garlic Butter Flavoured Bread Topped With Cheese

190

CHEESE CORN TIKKI

American Com & Cottage Cheese Patty Served with Mint Chutney

220

CRISPY CHILLY MUSHROOM

Crispy Fried Mushrooms Tossed With Oriental Spices

220

CRISPY CHILLY BABY CORN

Crispy Fried Babycom Tossed With Oriental Spices

190

CRISPY CORN CHILLY PEPPER

Fred Undi Lightly Crispy & Seasoned With Lime Juice, Salt, Chilli & Pepper

240

HONEY CHILLY POTATO

Crispy Fried Potatoes Tossed With Chilli And Honey

190"""

In [17]:
pages[2]="""$VEG STARTERS

SESAME CHILLI PANEER

Cottage Cheese Cubes Tossed In A Sweet & Spicy Sauce, Finished With Sesame Seeds

270

PANDAN LEAF COTTAGE CHEESE

Tender Cottage Cheese Wrapped In Fragrant Pandan Leaves, With Chilli Coriander Sauce

280

MEZZE BASKET

A Combination Of Hummus, Mutabal & Labneh, Served With Pita Bread

340

$FROM THE CLAY OVEN

MAKHMALI PANEER TIKKA

Mouth Melting Paneer Cubes Marinated With A Cheese Curd Mixture, Cream And Mild Spices

270

ANGARI PANEER TIKKA

Clay Oven Roasted Spicy Morsels Of Cottage Cheese

270

TANDOORI STUFFED MUSHROOMS

Mushrooms Filled With A Blend Of Indian Spices And Tangy Stuffing Mix, Adding A Unique Taste On The Bite

250

TANDOORI VEG MOMOS

Soft And Juicy Vegetable Momos Marinated With Tandoori Masala And Char Grilled

220

$CHICKEN STARTERS

CHICKEN LOADED FRIES

Cheesy French Fries Topped With Chicken & Herbs

260

SHANG DONG CHICKEN LOLLY POP

Crispy Lolly pops Crunched To Perfection & Tossed in our Signature Shang Dong Drip

260"""

In [18]:
pages[3]="""$CHICKEN STARTERS

HOC SIGNATURE CHICKEN WINGS

Sweet Chilly / Spiced Szechuan

260

PEPPER GARLIC CHICKEN

Cubes Of Chicken Tossed With Garlic And Pepper In Oriental Style

260

GUNTUR JUNCTION CHICKEN

Chicken Cubes Tossed With Freshly Ground Red Spices And Finished In South Indian Style

270

ANDHRA CHILLI CHICKEN *VERY HOT*

All Time Favourite Andhra Style Chicken Cooked With Coriander And Green Chilli Paste And Fresh Green Chillies

270

LEMON PEPPER CHICKEN

Slice Chicken Wok Tossed, Finished With Pepper Garlic & Lime

260

JIANG CHILLI CHICKEN

Chicken Tossed With Shredded Ginger And Chilli In Jiang Style

260

CHICKEN GHEE ROAST

Chicken Roasted With A Fiery Red, Tangy And Spicy With The Flavour Of Ghee

280

MASALA FRY CHICKEN (new)

Classic Indian Style With Bone Deep fried Chicken Served With Mint Chutney

250

$FROM THE CLAY OVEN

JAIPURI MURGH KA SOOLA

Succulent Pieces Of Chicken Zapped With Red Spices, Cinnamon Flavour & Finished In Tandoor

270

TANDOORI MURGH (HALF/FULL)

Chicken Marinated In Aromatic Indian Spices & Then Cooked In Tandoor

250/480

TANDOORI POMFRET

Indian Spices Marinated Whole Pomfret Skewered And Finished In Clay Oven

590"""

In [19]:
pages[4]="""$FROM THE CLAY OVEN

FISH TIKKA KALIMIRCH

Fresh Cubes Of Basa Fish Marinated With Crushed Black Pepper & Indian Spices, Finished In Clay Oven

320

AMRITSAR FISH TIKKA

Spicy Fish Tikka Marinated In Traditional Spices And Char Grilled

320

KALMI KEBAB (3pcs)

Clay Oven Roasted Chicken Drumsticks

260

ANGARI MURGH TIKKA

Clay Oven Roasted Spicy Morsels Of Chicken

260

TANDOORI LOLLIPOPS

Chicken Lollipops Marinated With Aromatic Indian Spices And Cooked In The Tandoor

270

MURGH MALAI TIKKA

Clay Oven Roasted Creamy Chunks Of Chicken

260

MURGH KASTURI KEBAB

Boneless Charcoal Smoked Chicken Tikka With Kasturi Methi Flavoured

260

TANDOORI CHICKEN MOMOS

Soft And Juicy Chicken Momos Marinated With Tandoori Masala And Char Grilled

260

TANDOORI KING PRAWNS

King Prawns Marinated With Yogurt, Lime Juice, Bhatti Spices And Slowly Cooked In Tandoor

499

$LAMB STARTERS

ANDHRA LAMB PEPPER FRY

Tender Mutton Pieces Tossed In South Indian Spices

350

MUTTON GHEE ROAST

Mutton Roasted With A Fiery Red And Spicy With The Flavour Of Ghee

370"""

In [20]:
pages[5]="""$LAMB STARTERS

LAHORI SEEKH KEBAB

Succulent Minced Lamb Kebabs In The True Lahori Street Food Style

370

RAILWAY MUTTON DRY

With Bone Mutton Cooked In Rich Indian Spices, As Served In The Railway Canteen Of The British Raj

350

$EGG STARTERS

EGG BHURJI

Desi Style Scrambled Eggs

170

SPICY MASALA OMELETTE

Slow Cooked Egg Omlet With Masala

180

EGG CHILLI 

Spicy Boiled Egg With Chilli Fry

210

$FROM THE GRILL

PERI PERI PANEER/CHICKEN

Peri Peri Basted Paneer / Chicken Cubes Served With Tzatziki Dip

260/280

THAI GRILLED FISH IN BANANA LEAF

Delicately Grilled Fish With Basil Flavour And Served Hot

330

HARISSA GRILLED FISH

Fish Marinated In A Spicy Tunisian Harissa Sauce And Grilled To Perfection

330

THAI GRILLED CHICKEN

Boneless Chicken Marinated With Thai Spices And Grilled

270"""

# Consolidating

## Pages to DF

In [21]:
half_the_menu=[]

for i in range(6):
    half_the_menu.append(pages[i])

half_menu_text="\n\n".join(half_the_menu)

In [22]:
str_menu = pd.DataFrame(columns=['sub_category','name','description','price'])

In [23]:
description=dish_name=price=subcat=''
counter=0
for line in half_menu_text.split("\n\n"):
    if(re.search("^[\$]",line)):
        subcat = line[1:].strip()
    else:
        if(counter%3==0):
            dish_name = line.strip()
        elif(counter%3==1):
            description = line.strip()
        elif(counter%3==2):
            price = line.strip()
            str_menu.loc[str_menu.shape[0]]=[subcat,dish_name,description,price]
        counter+=1

## creating main categories

In [24]:
main_category={"BAR BITES":"BAR BITES","SALADS":"SALADS","VEG STARTERS":"VEG STARTERS","CHICKEN STARTERS":"NON-VEG STARTERS","LAMB STARTERS":"NON-VEG STARTERS","EGG STARTERS":"NON-VEG STARTERS","FROM THE GRILL":"NON-VEG STARTERS","FROM THE CLAY OVEN":"CHANGE"}

In [25]:
str_menu.loc[:,"main_category"]=str_menu['sub_category'].apply(lambda x: main_category[x])

In [26]:
str_menu.loc[27,"main_category"]="VEG STARTERS"

str_menu.loc[28,"main_category"]="VEG STARTERS"

str_menu.loc[29,"main_category"]="VEG STARTERS"

str_menu.loc[30,"main_category"]="VEG STARTERS"

In [27]:
str_menu.loc[str_menu['main_category']=='CHANGE',"main_category"]="NON-VEG STARTERS"

In [28]:
str_menu.columns

Index(['sub_category', 'name', 'description', 'price', 'main_category'], dtype='object')

In [30]:
str_menu=str_menu[['main_category','sub_category', 'name', 'description', 'price']]

## Taking care of multiple prices

In [44]:
lengths = str_menu['price'].str.split('/').map(len)

str_menu = pd.DataFrame({'main_category': np.repeat(str_menu['main_category'], lengths),
                    'sub_category': np.repeat(str_menu['sub_category'], lengths),
                    'name': np.repeat(str_menu['name'],lengths),
                    'description': np.repeat(str_menu['description'],lengths),
                    'price': chainer(str_menu['price'])})

str_menu.reset_index(inplace=True,drop=True)

str_menu.loc[45,'price']="250"

pickle.dump(str_menu,open('half_menu_main.pkl','wb'))

str_menu=pickle.load(open('half_menu_main.pkl','rb'))

## COnverting into json

In [33]:
final_list_json=[]

In [35]:

for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['foodlist']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['foodlist'].append({})
            final_list_json[n]['sub_category'][p]['foodlist'][q]['name']= food_dict[1]['name']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['price']=food_dict[1]['price']
        
        

In [4]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

In [9]:
from pprint import pprint

In [1]:
json.load(Restaurant.objects[0].to_json())

NameError: name 'Restaurant' is not defined

# js.data

In [17]:
pickle.dump(final_json,open('jason.pkl','wb'))

In [11]:
pickle.dump(final_list_json,open('jason.pkl','wb'))